In [35]:
%pip install -r requirements.txt


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [36]:
!cp ./store.db ./store_copy.db

In [37]:
%load_ext sql
%sql sqlite:///store_copy.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [38]:
%%sql
PRAGMA foreign_keys=on;

 * sqlite:///store_copy.db
Done.


[]

## Задание 1
Провести промо-акцию: сделать скидку 50 центов на треки жанра Rock и 10% на треки формата FLAC. Если выполняются оба условия, то только скидка 10%. Округлять цены до центов. Вывести треки с 336-го по 361-й.

In [39]:
!cp ./store.db ./store_copy.db

In [40]:
%%sql

WITH tracks_to_be_discounted AS (
  SELECT
    tracks.trackid,
    tracks.mediatypeid,
    tracks.unitprice,
    genres.name genre,
    media_types.name mediatype
  FROM
    tracks
    JOIN genres USING(genreid)
    JOIN media_types USING(mediatypeid)
),
discounts AS (
  SELECT
    tracks_to_be_discounted.trackid,
    (CASE
      WHEN mediatype = "FLAC" THEN ROUND(unitprice * 0.90, 2) 
      WHEN genre = 'Rock' THEN ROUND(unitprice - 0.50, 2)
      ELSE unitprice
    END) AS new_price
    FROM tracks_to_be_discounted
)
UPDATE tracks
SET unitprice = (
  SELECT new_price
  FROM discounts
  WHERE tracks.trackid = discounts.trackid
);

SELECT *
FROM tracks
ORDER BY trackid ASC
LIMIT (361 - 336 + 1)
OFFSET 335;

 * sqlite:///store_copy.db
Done.
Done.


TrackId,Name,AlbumId,MediaTypeId,GenreId,Milliseconds,Bytes,UnitPrice
336,Latinha de Cerveja,29,1,9,166687,5532564,1.79
337,You Shook Me,30,1,1,315951,10249958,1.79
338,I Can't Quit You Baby,30,1,1,263836,8581414,2.69
339,Communication Breakdown,30,1,1,192653,6287257,1.79
340,Dazed and Confused,30,1,1,401920,13035765,0.89
341,The Girl I Love She Got Long Black Wavy Hair,30,1,1,183327,5995686,0.89
342,What is and Should Never Be,30,1,1,260675,8497116,0.89
343,Communication Breakdown(2),30,1,1,161149,5261022,0.89
344,Travelling Riverside Blues,30,1,1,312032,10232581,0.89
345,Whole Lotta Love,30,1,1,373394,12258175,0.89


## Задание 2
Добавить сегодняшний заказ на любую песню группы Green Day от клиента Dan	Miller. Вывести его.

In [41]:
!cp ./store.db ./store_copy.db

In [42]:
%%sql

WITH track AS (
  SELECT tracks.*
  FROM tracks
  JOIN albums USING(albumid)
  JOIN artists USING(artistid)
  WHERE artists.name = "Green Day"
  LIMIT 1
),
invoice AS (
  SELECT 
    customerid,
    (SELECT DATETIME("now")) invoicedate,
    address billingaddress,
    city billingcity,
    state billingstate,
    country billingcountry,
    postalcode billingpostalcode,
    (SELECT unitprice FROM track) total
  FROM customers
  WHERE firstname = "Dan" AND lastname = "Miller"
  LIMIT 1
)
INSERT INTO invoices (
  customerid,
  invoicedate,
  billingaddress,
  billingcity,
  billingstate,
  billingcountry,
  billingpostalcode,
  total
)
SELECT * FROM invoice;

-- Также добавляем строку в invoice_items
WITH track AS (
  SELECT tracks.*
  FROM tracks
  JOIN albums USING(albumid)
  JOIN artists USING(artistid)
  WHERE artists.name = "Green Day"
  LIMIT 1
),
invoice AS (
  SELECT *
  FROM invoices
  ORDER BY invoiceid DESC
  LIMIT 1
),
invoice_item AS (
  SELECT invoice.invoiceid,
  (SELECT trackid FROM track) trackid,
  1 quantity
  FROM invoice
)
INSERT INTO invoice_items (
  invoiceid,
  trackid,
  quantity
)
SELECT * FROM invoice_item;

SELECT *
FROM invoices
ORDER BY invoiceid DESC
LIMIT 1;

 * sqlite:///store_copy.db
Done.
Done.
Done.


InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
413,20,2024-11-26 09:54:46,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,1.99


## Задание 3
Заполнить столбец Total в таблице Invoices подходящими значениями. Вывести первые 10 заказов (id, date, total).

In [43]:
%%sql

WITH totals AS (
  SELECT
    invoiceid,
    ROUND(tracks.unitprice * invoice_items.quantity, 2) total
  FROM invoices
  JOIN invoice_items USING(invoiceid)
  JOIN tracks USING(trackid)
)
UPDATE invoices
SET total = (
  SELECT total
  FROM totals
  WHERE invoices.invoiceid = totals.invoiceid
);

SELECT invoiceid, invoicedate, total
FROM invoices
ORDER BY invoiceid ASC
LIMIT 10;


 * sqlite:///store_copy.db
Done.
Done.


InvoiceId,InvoiceDate,Total
1,2023-01-02 03:56:47,0.99
2,2023-01-03 03:43:33,6.98
3,2023-01-04 08:03:04,2.97
4,2023-01-07 23:29:33,0.99
5,2023-01-08 10:08:43,1.98
6,2023-01-08 13:25:49,2.97
7,2023-01-13 17:18:37,2.97
8,2023-01-16 17:43:11,5.98
9,2023-01-19 10:57:39,1.98
10,2023-01-22 20:21:33,4.95


## Задание 4
Создать таблицу Cities, где будут храниться названия городов, их областей и стран (City,	State,	Country). Сделать так, чтобы клиенты и сотрудники хранили только id города вместо 3 столбцов.

In [44]:
!cp ./store.db ./store_copy.db

In [45]:
%%sql

CREATE TABLE IF NOT EXISTS Cities (
  cityid INTEGER PRIMARY KEY AUTOINCREMENT,
  city TEXT NOT NULL,
  state TEXT,
  country TEXT NOT NULL
);

INSERT INTO cities (city, state, country)
SELECT DISTINCT city, state, country
FROM customers
WHERE city IS NOT NULL;

INSERT INTO cities (city, state, country)
SELECT DISTINCT e.city, e.state, e.country
FROM employees e
WHERE NOT EXISTS (
  SELECT city, state, country
  FROM cities
  WHERE
    city = e.city
    AND state = e.state
    AND country = e.country
);

ALTER TABLE customers ADD cityid INTEGER CONSTRAINT cityid_fk
REFERENCES cities (cityid) ON DELETE RESTRICT;

UPDATE customers
SET cityid = (
  SELECT cityid
  FROM cities
  WHERE cities.city = customers.city
  AND (cities.state = customers.state
    OR (cities.state IS NULL AND customers.state IS NULL))
  AND cities.country = customers.country
);

ALTER TABLE customers DROP COLUMN city;
ALTER TABLE customers DROP COLUMN state;
ALTER TABLE customers DROP COLUMN country;

ALTER TABLE employees ADD cityid INTEGER CONSTRAINT cityid_fk
REFERENCES cities (cityid) ON DELETE RESTRICT;

UPDATE employees
SET cityid = (
  SELECT cityid
  FROM cities
  WHERE cities.city = employees.city
  AND (cities.state = employees.state
    OR (cities.state IS NULL AND employees.state IS NULL))
  AND cities.country = employees.country
);

ALTER TABLE employees DROP COLUMN city;
ALTER TABLE employees DROP COLUMN state;
ALTER TABLE employees DROP COLUMN country;

SELECT * FROM cities LIMIT 5;

 * sqlite:///store_copy.db
Done.
53 rows affected.
2 rows affected.
Done.
59 rows affected.
Done.
Done.
Done.
Done.
8 rows affected.
Done.
Done.
Done.
Done.


cityid,city,state,country
1,São José dos Campos,SP,Brazil
2,Stuttgart,None,Germany
3,London,QC,Canada
4,Oslo,None,Norway
5,Prague,None,Czech Republic


## Задание 5
Объединить 3 таблицы - медиатипы, жанры и треки в одну общую таблицу tracks. Вывести результат (первые 10 треков).

In [46]:
!cp ./store.db ./store_copy.db

In [47]:
%%sql

CREATE TABLE IF NOT EXISTS tracks_extended AS
SELECT
    t.trackid,
    t.name,
    t.albumid,
    m.name AS mediatype,
    g.name AS genre,
    t.milliseconds,
    t.bytes,
    t.unitprice
FROM tracks t
JOIN genres g USING(genreid)
JOIN media_types m USING(mediatypeid);

PRAGMA foreign_keys=off;

DROP TABLE tracks;
ALTER TABLE tracks_extended RENAME TO tracks;

PRAGMA foreign_keys=on;

SELECT * FROM tracks LIMIT 10;
-- FOREIGN KEY отношение сохранено, например в таблице playlist_track
SELECT * FROM playlist_track JOIN tracks USING(trackid) LIMIT 10;

 * sqlite:///store_copy.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.


PlaylistId,TrackId,Name,AlbumId,mediatype,genre,Milliseconds,Bytes,UnitPrice
3,3250,Pilot,254,MPEG audio file,TV Shows,2484567,492670102,1.99
3,2819,Battlestar Galactica: The Story So Far,226,MPEG audio file,Science Fiction,2622250,490750393,1.99
3,2820,Occupation / Precipice,227,MPEG audio file,TV Shows,5286953,1054423946,1.99
3,2821,"Exodus, Pt. 1",227,MPEG audio file,TV Shows,2621708,475079441,1.99
3,2822,"Exodus, Pt. 2",227,MPEG audio file,TV Shows,2618000,466820021,1.99
3,2823,Collaborators,227,MPEG audio file,TV Shows,2626626,483484911,1.99
3,2824,Torn,227,MPEG audio file,TV Shows,2631291,495262585,1.99
3,2825,A Measure of Salvation,227,MPEG audio file,Science Fiction,2563938,489715554,1.99
3,2826,Hero,227,MPEG audio file,Science Fiction,2713755,506896959,1.99
3,2827,Unfinished Business,227,MPEG audio file,Science Fiction,2622038,528499160,1.99
